In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ydata_profiling import ProfileReport
from movie_recommender_system import paths
import pandas as pd

In [ ]:
movie_data_path = paths.data_raw_dir() / "tmdb_5000_movie_dataset/movies.csv"
credits_data_path = paths.data_raw_dir() / "tmdb_5000_movie_dataset/credits.csv"

In [ ]:
df_movies = pd.read_csv(movie_data_path)
profile_movies = ProfileReport(df_movies, title="Profiling Report")
profile_movies.to_file(paths.reports_dir() / "00_raw_movies.html")

In [ ]:
df_credits = pd.read_csv(credits_data_path)
profile_credits = ProfileReport(df_credits, title="Profiling Report")
profile_credits.to_file(paths.reports_dir() / "00_raw_credits.html")

In [ ]:
df_dataset = df_movies.merge(df_credits, on="title")

In [ ]:
list(df_dataset.columns)

In [ ]:
df_dataset = df_dataset[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
df_dataset.sample(5, random_state=1)

In [ ]:
df_dataset.isnull().sum()

In [ ]:
df_dataset_valid = df_dataset.dropna()

In [ ]:
import ast

def extract_data_from_dict_string(string_dict, key) -> list:
    value_list = []
    for item in ast.literal_eval(string_dict):
        value_list.append(item[key])
    return value_list

In [ ]:
df_dataset_valid.loc[:, 'genres'] = df_dataset_valid['genres'].apply(lambda x: extract_data_from_dict_string(x, 'name'))
df_dataset_valid.loc[:, 'keywords'] = df_dataset_valid['keywords'].apply(lambda x: extract_data_from_dict_string(x, 'name'))


In [ ]:
df_dataset_valid.sample(5, random_state=1)

In [ ]:
ast.literal_eval(df_dataset_valid["cast"][0])[0]

In [ ]:
ast.literal_eval(df_dataset_valid["crew"][0])

In [ ]:
def extract_cast(string_dict: str, key: str, count: int) -> list:
    value_list = []
    counter = 0
    for item in ast.literal_eval(string_dict):
        if counter == count:
            break
        else:
            value_list.append(item[key])
            counter += 1
    return value_list

In [ ]:
cast_count = 3
df_dataset_valid.loc[:, 'cast'] = df_dataset_valid['cast'].apply(lambda x: extract_cast(x, 'name', cast_count))

In [ ]:
def extract_crew(string_dict: str, key: str, crew_jobs: list) -> list:
    value_list = []
    for item in ast.literal_eval(string_dict):
        if item['job'] in crew_jobs:
            value_list.append(item[key])
    return value_list

In [ ]:
crew_jobs_list = ['Director', 'Producer', 'Screenplay']
df_dataset_valid.loc[:, 'crew'] = df_dataset_valid['crew'].apply(lambda x: extract_crew(x, 'name', crew_jobs_list))

In [ ]:
df_dataset_valid.sample(5, random_state=1)

In [ ]:
def remove_space(string_list: list) -> list:
    return [x.replace(" ", "") for x in string_list]

df_dataset_valid.loc[:, 'genres'] = df_dataset_valid['genres'].apply(lambda x: remove_space(x))